### Methodolody

What I want to do is look at the practice sessions for each race and look at the long runs of each car to find the tyre deg, then apply that and work through lap by lap for the race.

Then obviously the pit stops are whatever has the lowest total race time.

This could be put futher into even looking at other cars and not pitting into traffic, but then we start getting quite complicated.

Really just want to do this as a proof of concept, then look into ML models. May come back and improve to compare to ML models.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap

if tf.config.list_physical_devices('GPU'):
    print("TensorFlow is using GPU!")
else:
    print("TensorFlow is not using GPU.")


TensorFlow is not using the GPU.


## Collect data into df

In [17]:
def create_dataframe():
    # initialize db connection and session
    db_engine, db_session = init_db()

    # query data from the database
    query = db_session.query(
        RacingWeekend.year,
        RacingWeekend.round,
        Circuit.circuit_name,
        Driver.driver_name,
        Driver.driver_short,
        Lap.lap_num,
        Lap.lap_time,
        Lap.tyre,
        Lap.pit,
        Session.session_type
    ).join(RacingWeekend.circuit) \
     .join(RacingWeekend.sessions) \
     .join(Session.laps) \
     .join(Lap.driver) \
     .join(RacingWeekend.season) \
     .all()

    # convert result to list of dicts
    data = []
    for row in query:
        data.append({
            'year': row.year,
            'round': row.round,
            'circuit_name': row.circuit_name,
            'driver_name': row.driver_name,
            'driver_short': row.driver_short,
            'lap_num': row.lap_num,
            'lap_time': row.lap_time,
            'tyre': row.tyre,
            'pit': row.pit,
            'session_type': row.session_type
        })

    # create dataframe
    df = pd.DataFrame(data)

    # encode 'tyre' and 'pit' as categories
    # df['tyre'] = df['tyre'].astype('category')
    # df['pit'] = df['pit'].astype('category')

    # one-hot encode categorical variables
    df = pd.get_dummies(df, columns=['circuit_name', 'driver_name', 'driver_short', 'session_type'], drop_first=True)

    return df

# create the dataframe and print it
df = create_dataframe()

print(df.columns.tolist())




['year', 'round', 'lap_num', 'lap_time', 'tyre', 'pit', 'circuit_name_Baku', 'circuit_name_Barcelona', 'circuit_name_Budapest', 'circuit_name_Hockenheim', 'circuit_name_Imola', 'circuit_name_Istanbul', 'circuit_name_Jeddah', 'circuit_name_Las Vegas', 'circuit_name_Le Castellet', 'circuit_name_Lusail', 'circuit_name_Marina Bay', 'circuit_name_Melbourne', 'circuit_name_Mexico City', 'circuit_name_Miami', 'circuit_name_Monaco', 'circuit_name_Monte Carlo', 'circuit_name_Montréal', 'circuit_name_Monza', 'circuit_name_Mugello', 'circuit_name_Nürburgring', 'circuit_name_Portimão', 'circuit_name_Sakhir', 'circuit_name_Shanghai', 'circuit_name_Silverstone', 'circuit_name_Singapore', 'circuit_name_Sochi', 'circuit_name_Spa-Francorchamps', 'circuit_name_Spielberg', 'circuit_name_Suzuka', 'circuit_name_São Paulo', 'circuit_name_Yas Island', 'circuit_name_Zandvoort', 'driver_name_Alexander Albon', 'driver_name_Andrea Kimi Antonelli', 'driver_name_Antonio Giovinazzi', 'driver_name_Arthur Leclerc', '

In [ ]:
race_year = 2022
race_round = 10

# filter training data
train_data = df[
    (df['year'] < race_year) |
    ((df['year'] == race_year) & (
        (df['round'] < race_round) |
        ((df['round'] == race_round) & (df['session_type_Race'] == 0))  # practice/qualifying of the selected round
    ))
]

# filter testing data (only race session)
test_data = df[
    (df['year'] == race_year) &
    (df['round'] == race_round) &
    (df['session_type_Race'] == 1)  # race session only
]

# define features (X) and targets (y) for training
X_train = train_data.drop(columns=['pit', 'tyre', 'lap_time'])
y_train = train_data[['pit', 'tyre']]

# define features (X) and targets (y) for testing
X_test = test_data.drop(columns=['pit', 'tyre', 'lap_time'])
y_test = test_data[['pit', 'tyre']]

y_train = y_train.astype(int)
y_test = y_test.astype(int)
# Feature scaling (LSTM models work better with normalized data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape data for LSTM (samples, time_steps, features)
# treat each lap as a time step.
# reshape X_train and X_test to have a time step dimension (e.g., 1 lap at a time).

X_train_lstm = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_test_lstm = X_test_scaled.reshape((X_test_scaled.shape[0], 1, X_test_scaled.shape[1]))

model = Sequential()

# add LSTM layer
model.add(LSTM(64, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2]), return_sequences=False))
model.add(Dropout(0.2))

# add Dense layer for output
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))  # 2 output nodes for 'pit' and 'tyre'

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# train the LSTM model
model.fit(X_train_lstm, y_train, epochs=10, batch_size=32, validation_data=(X_test_lstm, y_test))

# predict on the test data
y_pred_test = model.predict(X_test_lstm)

# convert predictions to binary class labels
y_pred_pit = (y_pred_test[:, 0] > 0.5).astype(int)
y_pred_tyre = (y_pred_test[:, 1] > 0.5).astype(int)

# add predictions to the test data
test_data['predicted_pit'] = y_pred_pit
test_data['predicted_tyre'] = y_pred_tyre

# calculate accuracy for pit and tyre predictions
accuracy_pit = accuracy_score(y_test['pit'], y_pred_pit)
accuracy_tyre = accuracy_score(y_test['tyre'], y_pred_tyre)

print(f"Accuracy (Pit): {accuracy_pit:.2f}")
print(f"Accuracy (Tyre): {accuracy_tyre:.2f}")

# confusion matrix for pit and tyre predictions
conf_matrix_pit = confusion_matrix(y_test['pit'], y_pred_pit)
conf_matrix_tyre = confusion_matrix(y_test['tyre'], y_pred_tyre)

print("\nConfusion Matrix (Pit):")
print(conf_matrix_pit)

print("\nConfusion Matrix (Tyre):")
print(conf_matrix_tyre)

# classification report for pit and tyre predictions
class_report_pit = classification_report(y_test['pit'], y_pred_pit)
class_report_tyre = classification_report(y_test['tyre'], y_pred_tyre)

print("\nClassification Report (Pit):")
print(class_report_pit)

print("\nClassification Report (Tyre):")
print(class_report_tyre)

/home/ben/individual-project-BenMcD23/env/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


2024-12-10 12:03:32.951050: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 96302768 exceeds 10% of free system memory.


5154/5154 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.9596 - loss: -2569.6531 - val_accuracy: 0.9909 - val_loss: -20508.5391
Epoch 2/100
5154/5154 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9666 - loss: -28345.6387 - val_accuracy: 0.9909 - val_loss: -67244.9766
Epoch 3/100
5154/5154 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9662 - loss: -76634.1797 - val_accuracy: 0.9909 - val_loss: -137370.3125
Epoch 4/100
5154/5154 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9662 - loss: -147436.1719 - val_accuracy: 0.9909 - val_loss: -231151.3750
Epoch 5/100
5154/5154 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9665 - loss: -240912.9062 - val_accuracy: 0.9909 - val_loss: -348447.3438
Epoch 6/100
5154/5154 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9656 - loss: -353088.7188 - val_accuracy: 0.9909 - val_loss: -488948.5000
Epoch 7/100
5154/5154 ━━━━━━━━━━━━━━━━━━━━ 10s 2ms/step - accuracy: 0.9670 - loss: -492406.7188 - val_accuracy: 0.9909 - val_loss: -652419.0000
Epoch 8/10

/tmp/ipykernel_50985/1668913275.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_pit'] = y_pred_pit
/tmp/ipykernel_50985/1668913275.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['predicted_tyre'] = y_pred_tyre
/home/ben/individual-project-BenMcD23/env/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control 

In [ ]:
# define tyre mapping
tyre_mapping = {
    1: 'SOFT',
    2: 'MEDIUM',
    3: 'HARD',
    4: 'INTERMEDIATE',
    5: 'WET',
}

# function to decode tyre value from integer
def decode_tyres(value):
    return tyre_mapping.get(value, None)  # return None if no matching tyre value

# function to extract driver name based on driver_short columns
def get_driver_name(row):
    for col in test_data.columns:
        if col.startswith('driver_short_') and row[col] == 1:
            return col.replace('driver_short_', '')  # Extract driver name
    return None

# add driver names to the test data
test_data['driver_name'] = test_data.apply(get_driver_name, axis=1)

# get the laps where any driver actually pitted
actual_pit_laps = test_data[test_data['pit'] == 1][['lap_num', 'driver_name', 'pit', 'tyre']]

# get the laps where the model predicted a pit stop
predicted_pit_laps = test_data[test_data['predicted_pit'] == 1][['lap_num', 'driver_name', 'predicted_pit', 'predicted_tyre']]

# apply the decoding function to actual and predicted pit laps
actual_pit_laps['decoded_tyre'] = actual_pit_laps['tyre'].apply(decode_tyres)
predicted_pit_laps['decoded_tyre'] = predicted_pit_laps['predicted_tyre'].apply(decode_tyres)

# group by driver and sort pit stops by lap number
actual_pit_laps_sorted = actual_pit_laps.sort_values(by=['driver_name', 'lap_num'])
predicted_pit_laps_sorted = predicted_pit_laps.sort_values(by=['driver_name', 'lap_num'])

print(f"Actual Pit Stops in Race {race_year} Round {race_round}:")
for driver, group in actual_pit_laps_sorted.groupby('driver_name'):
    print(f"\n{driver}'s Actual Pit Stops:")
    print(group[['lap_num', 'decoded_tyre']])

# Print predicted pit stops for all drivers
print(f"\nPredicted Pit Stops in Race {race_year} Round {race_round}:")
for driver, group in predicted_pit_laps_sorted.groupby('driver_name'):
    print(f"\n{driver}'s Predicted Pit Stops:")
    print(group[['lap_num', 'decoded_tyre']])


Actual Pit Stops in Race 2022 Round 10:

ALO's Actual Pit Stops:
        lap_num decoded_tyre
165057       34         HARD
165063       40         SOFT

GAS's Actual Pit Stops:
        lap_num decoded_tyre
164966       17         HARD

HAM's Actual Pit Stops:
        lap_num decoded_tyre
165442       34         HARD

LAT's Actual Pit Stops:
        lap_num decoded_tyre
165623       20       MEDIUM
165643       40         SOFT

LEC's Actual Pit Stops:
        lap_num decoded_tyre
165099       26         HARD

MAG's Actual Pit Stops:
        lap_num decoded_tyre
165195       23       MEDIUM

NOR's Actual Pit Stops:
        lap_num decoded_tyre
165394       35         HARD

OCO's Actual Pit Stops:
        lap_num decoded_tyre
165346       23         HARD

PER's Actual Pit Stops:
        lap_num decoded_tyre
164980        6       MEDIUM
165014       40         SOFT

RIC's Actual Pit Stops:
        lap_num decoded_tyre
165294       21         HARD
165306       33         SOFT

SAI's Actual 

/tmp/ipykernel_50985/404487195.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['driver_name'] = test_data.apply(get_driver_name, axis=1)
